<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#scrap-teams-and-players" data-toc-modified-id="scrap-teams-and-players-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>scrap teams and players</a></span></li><li><span><a href="#scrap-matches" data-toc-modified-id="scrap-matches-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>scrap matches</a></span></li><li><span><a href="#scrap-goals-par-player/match" data-toc-modified-id="scrap-goals-par-player/match-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>scrap goals par player/match</a></span></li><li><span><a href="#weather" data-toc-modified-id="weather-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>weather</a></span></li><li><span><a href="#add-df-to-sql-database" data-toc-modified-id="add-df-to-sql-database-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>add df to sql database</a></span></li></ul></div>

In [1]:
import pandas as pd
import matplotlib.pyplot  as plt
import seaborn as sns

In [2]:
url = "https://www.lequipe.fr/Football/"

In [3]:
# function to get page soup from html page
import requests
from bs4 import BeautifulSoup as soup

from urllib.request import urlopen as uReq
def get_page_soup(url):
    # opening connection
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()
    #html parser
    return soup(page_html, "html.parser")


In [4]:
#get championships links
main_soup = get_page_soup(url)
#equipe_soup
#a aria-label = '/Football/' class = 'Link Menu__accordionTitle'
championships = main_soup.find("div", {"class": 'Menu__accordion js-menu-accordion is-active'}).findAll("li", {"class":"Menu__item" })
print(championships[0].a['href'])
championships_links = {champ.a.text.split('\n')[1].strip(): champ.a['href'] for champ in championships}
championships_links

https://www.lequipe.fr/Football/Euro/


{'Euro': 'https://www.lequipe.fr/Football/Euro/',
 'Copa America': 'https://www.lequipe.fr/Football/copa-america/page-calendrier-resultats',
 'Ligue 1': 'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats',
 'Ligue 2': 'https://www.lequipe.fr/Football/ligue-2/page-calendrier-resultats',
 'Ligue des Champions': 'https://www.lequipe.fr/Football/ligue-des-champions/page-calendrier-resultats',
 'Ligue Europa': 'https://www.lequipe.fr/Football/ligue-europa/page-calendrier-resultats',
 'Angleterre': 'https://www.lequipe.fr/Football/championnat-d-angleterre/page-calendrier-resultats',
 'Espagne': 'https://www.lequipe.fr/Football/championnat-d-espagne/page-calendrier-resultats',
 'Italie': 'https://www.lequipe.fr/Football/championnat-d-italie/page-calendrier-resultats',
 'Allemagne': 'https://www.lequipe.fr/Football/championnat-d-allemagne/page-calendrier-resultats',
 'Coupe de France': 'https://www.lequipe.fr/Football/coupe-de-france/page-calendrier-resultats',
 'Euro Espoirs':

In [5]:
championship = get_page_soup(championships_links['Ligue 1'])
#print(championships_links['Ligue 1'])
navigator_items = championship.find("div", {"class": "MainNav__items js-mainNavItems"}).findAll("a", {"class": "Link MainNav__item"})

results_link = ""
teams_link = ""
matches_link = ""
print('navigation items:')
for item in navigator_items:
    print(item.text)
    
for item in navigator_items:

    if "résultats" in item.text.strip().split():
        print("res")
        print(item["href"])
    if "sélections" in item.text.strip().split() or "clubs" in item.text.strip().split():
        print('TEAMS link: {}'.format(item["href"]))   
    if "matches" in item.text.strip().split() or "matchs" in item.text.strip().split():
        print('MATCHES link: {}'.format(item["href"]))
        
print(results_link)
print(teams_link)
print(matches_link)
    

navigation items:
actus 
classement 
barrages L1/L2 
les notes 
buteurs 
passeurs 
fairplay 
les clubs 
tous les matches 
palmarès 
archives Ligue 1 
calendrier présaison 
Botaldo 
TEAMS link: https://www.lequipe.fr/Football/ligue-1/page-participants
MATCHES link: https://www.lequipe.fr/Football/ligue-1/page-calendrier-general





In [7]:
for champ in championships_links.keys():
    championship = get_page_soup(championships_links[champ])
    navigator_items = championship.find("div", {"class": "MainNav__items js-mainNavItems"}).findAll("a", {"class": "Link MainNav__item"})
    print(champ)
    print('navigation items:')
    for item in navigator_items:
        print(item.text)

    
    results_link = ""
    teams_link = ""
    matches_link = ""
    
    for item in navigator_items:
        if "résultats" in item.text.strip().split():
            results_link = item["href"]
        if "sélections" in item.text.strip().split() or "clubs" in item.text.strip().split():
            teams_link = item["href"]   
        if "matches" in item.text.strip().split() or "matchs" in item.text.strip().split():
            matches_link = item["href"]
            
    #result_soup = get_page_soup(results_link) 
    #name_championship = result_soup.find("h1", {"class": "heading heading--1"}).text
    
    #print(name_championship)
    print('RESULT link:'.format(results_link))
    print('TEAMS link: {}'.format(teams_link)) 
    print('MATCHES link: {}'.format(matches_link))
    
    print("\n")

    
    

Euro
navigation items:
phase finale 
calendrier & résultats (groupes) 
groupe F (France) 
tous les matchs 
buteurs 
passeurs 
fairplay 
les sélections 
class. FIFA 
le guide de l'euro 
vos pronostics 
calendrier TV de l'Euro 
palmarès 
archives Euro 
RESULT link:
TEAMS link: https://www.lequipe.fr/Football/euro/saison-2020/page-participants
MATCHES link: https://www.lequipe.fr/Football/euro/page-calendrier-general


Copa America
navigation items:
actus 
buteurs 
les sélections 
tous les matches 
fairplay 
palmarès 
archives Copa America 
RESULT link:
TEAMS link: https://www.lequipe.fr/Football/copa-america/page-participants
MATCHES link: https://www.lequipe.fr/Football/copa-america/page-calendrier-general


Ligue 1
navigation items:
actus 
classement 
barrages L1/L2 
les notes 
buteurs 
passeurs 
fairplay 
les clubs 
tous les matches 
palmarès 
archives Ligue 1 
calendrier présaison 
Botaldo 
RESULT link:
TEAMS link: https://www.lequipe.fr/Football/ligue-1/page-participants
MATCHES lin

Belgique
navigation items:
actus 
classement 
buteurs 
les clubs 
tous les matches 
fairplay 
palmarès 
archives 
RESULT link:
TEAMS link: https://www.lequipe.fr/Football/championnat-de-belgique/page-participants
MATCHES link: https://www.lequipe.fr/Football/championnat-de-belgique/page-calendrier-general


Brésil
navigation items:
actus 
classement 
les clubs 
tous les matches 
palmarès 
archives 
RESULT link:
TEAMS link: https://www.lequipe.fr/Football/championnat-du-bresil/page-participants
MATCHES link: https://www.lequipe.fr/Football/championnat-du-bresil/page-calendrier-general


Écosse
navigation items:
actus 
classement 
buteurs 
les clubs 
tous les matches 
fairplay 
palmarès 
RESULT link:
TEAMS link: https://www.lequipe.fr/Football/championnat-d-ecosse/page-participants
MATCHES link: https://www.lequipe.fr/Football/championnat-d-ecosse/page-calendrier-general


Grèce
navigation items:
classement 
les clubs 
tous les matches 
palmares 
archives 
RESULT link:
TEAMS link: https:

IncompleteRead: IncompleteRead(579031 bytes read)

### scrap teams and players



In [8]:
## create database teams

In [9]:
teams_page = get_page_soup("https://www.lequipe.fr/Football/ligue-1/page-participants")

teams = teams_page.find("ul", {"class": "clubs"}).findAll("li")

teams_links = {'team_short_names': [], 'links': []}
for team in teams:
    if team.h2 is not None:
        teams_links['team_short_names'].append(team.h2.a.text.strip())
        teams_links['links'].append(team.h2.a['href'])
        #teams_link[team.h2.a.text.strip()] = team.h2.a['href']

# print(teams[0].h2.a.text.strip())
# print(teams[0].h2.a['href'])
    
df_teams_links = pd.DataFrame(teams_links)
df_teams_links.links = 'https://www.lequipe.fr'+df_teams_links.links
df_teams_links.head()

,team_short_names,links
0,Angers,https://www.lequipe.fr/Football/FootballFicheC...
1,Bordeaux,https://www.lequipe.fr/Football/FootballFicheC...
2,Brest,https://www.lequipe.fr/Football/FootballFicheC...
3,Dijon,https://www.lequipe.fr/Football/FootballFicheC...
4,Lens,https://www.lequipe.fr/Football/FootballFicheC...


In [10]:
# get info for one team
dict_teams = {'club': [], 'full_name': [], 'coach': [], 'president': [], 'foundation': []}
dict_players = {'number': [], 'name': [], 'country': [], 'birth': [], 'position': [], 'club': []}
df_players = pd.DataFrame(dict_players)
#print(df_players.head())
for i in range(len(df_teams_links)):
    
    link = df_teams_links.loc[i,'links']
    name = df_teams_links.loc[i,'team_short_names']
    #print(name)
    # team info and player info dirty tables
    team_info = pd.read_html(link)[0]
    
    dict_teams['club'].append(name)
    dict_teams['full_name'].append(team_info.loc[0,0].split(':')[1].strip())
    dict_teams['foundation'].append(team_info.loc[1,0].split(':')[1].strip())
    dict_teams['president'].append(team_info.loc[2,0].split(':')[1].strip())
    dict_teams['coach'].append(team_info.loc[3,0].split(':')[1].strip())
    
    # players info
    if len(pd.read_html(link)[2])>5:
        team_players = pd.read_html(link)[2]  
    else:
        team_players = pd.read_html(link)[1]  
        
    old_column_names = team_players.columns
    team_players = team_players.rename(columns={'Unnamed: 0': 'number', 'Nom': 'name', 'Pays': 'country', old_column_names[3]: 'birth', 'Pos.': 'position'})
    team_players['club']= name 

    df_players = pd.concat([df_players, team_players],ignore_index = True)
    #print(df_players.tail(2))
    
    
    
df_teams = pd.DataFrame(dict_teams) 


In [11]:
df_teams.head(3)

,club,full_name,coach,president,foundation
0,Angers,Angers Sporting Club de l'Ouest,Gérald BATICLE,Saïd Chabane,1919
1,Bordeaux,Football Club des Girondins de Bordeaux,Jean-Louis GASSET,Frédéric Longuépée,1881
2,Brest,Stade Brestois 29,Michel DER ZAKARIAN,Denis Le Saint,1950


In [12]:
df_players.head(3)

,number,name,country,birth,position,club
0,1.0,P. Bernardoni,FRA,18/04/97,Gar.,Angers
1,16.0,L. Butelle,FRA,3/04/83,Gar.,Angers
2,30.0,D. Petkovic,MNE,25/05/93,Gar.,Angers


### scrap matches 

In [13]:
link_matches = 'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats'
matches_page = get_page_soup(link_matches)

matches_json = []
matches = matches_page.find("select",{"class":"SelectNav__select"}).findAll('option')
for match in matches:
    matches_json.append(match['value'])

In [14]:
#data from one json table

dict_matches = {
    'date': [],
    'lieu': [],
    'team_domicile': [],
    'team_exterieur': [],
    'score_domicile': [],
    'score_exterieur': [],
    'vainqueur': []
}


import json
import requests

for match_json in matches_json:
    response = requests.get(match_json)
    data = json.loads(response.text)
    for day in data['items'][:-1]:
        for team in day['items']:
            dict_matches["lieu"].append(team['event']['lieu']['ville'])
            dict_matches["date"].append(team['date'])
            dict_matches["team_domicile"].append(team['event']['specifics']['domicile']['equipe'][
                'nom'])
            dict_matches["team_exterieur"].append(team['event']['specifics']['exterieur']['equipe'][
                'nom'])
            dict_matches["score_domicile"].append(team['event']['specifics']['score']['domicile'])
            dict_matches["score_exterieur"].append(team['event']['specifics']['score']['exterieur'])
            dict_matches["vainqueur"].append(team['event']['specifics']['vainqueur'])

            #print("Stade:{} Date:{}".format(lieu, date))
            #print("{} {}:{} {}".format(team_domicile, score_domicile, score_exterieur, team_exterieur))
            #print("\n")
            
            
df_matches = pd.DataFrame(dict_matches)

In [15]:
df_matches

,date,lieu,team_domicile,team_exterieur,score_domicile,score_exterieur,vainqueur
0,2020-08-21T19:00:00+0200,Bordeaux,Bordeaux,Nantes,0,0,ex-aequo
1,2020-08-22T17:00:00+0200,Dijon,Dijon,Angers,0,1,exterieur
2,2020-08-22T21:00:00+0200,Lille,Lille,Rennes,1,1,ex-aequo
3,2020-08-23T13:00:00+0200,Monaco,Monaco,Reims,2,2,ex-aequo
4,2020-08-23T15:00:00+0200,Lorient,Lorient,Strasbourg,3,1,domicile
...,...,...,...,...,...,...,...
375,2021-05-23T21:00:00+0200,Rennes,Rennes,Nîmes,2,0,domicile
376,2021-05-23T21:00:00+0200,Nantes,Nantes,Montpellier,1,2,exterieur
377,2021-05-23T21:00:00+0200,Saint-Étienne,Saint-Étienne,Dijon,0,1,exterieur
378,2021-05-23T21:00:00+0200,Reims,Reims,Bordeaux,1,2,exterieur


In [16]:
## create database

### scrap goals par player/match

In [31]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import csv
import pandas as pd
import requests
import re
import json
from http.client import IncompleteRead
import numpy as np

liste = []
for i in np.arange(1,39):
    if i == 1 :
        url2 = f'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/{i}re-journee'
        
        
    else: 
        url2 = f'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/{i}e-journee'   


    page = urllib.request.urlopen(url2)
    page_soup = BeautifulSoup(page, 'html.parser')
    matches_json = []
    main = page_soup.find('div', attrs={'class':'container__main'})
    teams_html = main.findAll('div', attrs={'class' : 'TeamScore__data'})
    for l in teams_html:
        link_teams = l.find('a', attrs={'class':'Link'}, href=True)
        a = link_teams['href']
        liste.append(a)


In [32]:
liste_g = []
liste_d = []
for z in liste :
         
        url_marqueur = z

        page1 = urllib.request.urlopen(url_marqueur,timeout=5)
        soup1 = BeautifulSoup(page1, 'html.parser')

        main_mark = soup1.findAll('div', attrs={'class':'grid__content Scoreboard__goalList'})

        list_dld = []
        list_dlg = []

        if main_mark == []:

            liste_g.append([])
            liste_d.append([])

        else:

            for s in main_mark[0].findAll('span', attrs={'class' : 'Scoreboard__scorer'}):

                list_dlg.append(s.text)


            for s in main_mark[1].findAll('span', attrs={'class' : 'Scoreboard__scorer'}):     

                list_dld.append(s.text)



            liste_g.append(list_dlg)
            liste_d.append(list_dld)
            
       


In [33]:
# create database
liste_g

[[],
 ['I. Traoré'],
 ['D. Da Silva'],
 ['B. Dia', 'E. Touré'],
 ['M. Chahiri'],
 [],
 ['G. Kakuta'],
 ['M. Depay'],
 [],
 ['R. Hamouma', 'D. Bouanga'],
 ['A. Scheidler'],
 ['G. Laborde'],
 ['K. Dolberg', 'K. Dolberg'],
 ['J. Bamba'],
 ['Z. Ferhat'],
 [],
 ['J. Maja', 'T. Basic'],
 ['B. Badiashile'],
 ['F. Thauvin', 'D. Caleta-Car', 'D. Caleta-Car'],
 [],
 [],
 ['Dante'],
 [],
 [],
 ['S. Guirassy', 'S. Guirassy', 'N. Aguerd', 'B. Bourigeaud'],
 ['G. Kakuta', 'F. Medina', 'I. Ganago'],
 [],
 ['R. Perraud', 'I. Cardona'],
 ['L. Blas'],
 ['F. Thauvin'],
 [],
 ['S. Kalu'],
 ['W. Ben Yedder'],
 ['K. Mbappé', 'A. Di Maria', 'Marquinhos'],
 ['S. Bahoken'],
 ['C. Hérelle', 'T. Monconduit'],
 ['B. Dia'],
 [],
 ['A. Aouchiche', 'Y. Maçon'],
 ['L. Araujo'],
 [],
 ['N. Aguerd', 'S. Guirassy', 'A. Hunou'],
 ['I. Niane'],
 [],
 ['A. Delort', 'T. Savanier'],
 ['I. Ganago'],
 ['M. Chahiri', 'L. Ajorque'],
 ['S. Mounié', 'G. Charbonnier'],
 ['L. Dubois'],
 ['M. Icardi', 'M. Icardi'],
 ['I. Traoré'],
 [

In [34]:
df_matches['goals_domicile'] = liste_g

In [35]:
df_matches['goals_exterieur'] = liste_d

In [37]:
df_matches.tail()

,date,lieu,team_domicile,team_exterieur,score_domicile,score_exterieur,vainqueur,goals_domicile,goals_exterieur
375,2021-05-23T21:00:00+0200,Rennes,Rennes,Nîmes,2,0,domicile,"[S. Guirassy, B. Bourigeaud]",[]
376,2021-05-23T21:00:00+0200,Nantes,Nantes,Montpellier,1,2,exterieur,[R. Kolo Muani],"[G. Laborde, A. Delort]"
377,2021-05-23T21:00:00+0200,Saint-Étienne,Saint-Étienne,Dijon,0,1,exterieur,[],[A. Kamara]
378,2021-05-23T21:00:00+0200,Reims,Reims,Bordeaux,1,2,exterieur,[E. Touré],"[Y. Adli, E. Kwateng]"
379,2021-05-23T21:00:00+0200,Strasbourg,Strasbourg,Lorient,1,1,ex-aequo,[H. Diallo],[T. Chalobah]


### weather 

In [ ]:
# add to data base matches

{"message":"You are not subscribed to this API."}


### add df to sql database 